In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32


In [3]:
# Define data generators for train, validation and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)
train_generator =train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)


val_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE, IMG_SIZE),
     batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_datagen =ImageDataGenerator(rescale=1./255)

test_generator =test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection/test',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
)


Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 60 images belonging to 1 classes.


In [4]:
#Define the model
model = keras.Sequential([
    layers.Conv2D(32, (3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="sigmoid"),
  ])

In [5]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])


In [11]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
75/75 [==============================] - 376s 5s/step - loss: 0.5445 - accuracy: 0.7629 - val_loss: 0.3684 - val_accuracy: 0.8367
Epoch 2/5
75/75 [==============================] - 296s 4s/step - loss: 0.2576 - accuracy: 0.9008 - val_loss: 0.1994 - val_accuracy: 0.9067
Epoch 3/5
75/75 [==============================] - 298s 4s/step - loss: 0.1419 - accuracy: 0.9508 - val_loss: 0.2663 - val_accuracy: 0.9083
Epoch 4/5
75/75 [==============================] - 270s 4s/step - loss: 0.1111 - accuracy: 0.9625 - val_loss: 0.0424 - val_accuracy: 0.9833
Epoch 5/5
75/75 [==============================] - 292s 4s/step - loss: 0.0408 - accuracy: 0.9887 - val_loss: 0.0150 - val_accuracy: 0.9967


In [6]:
model.save("model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
#test your image
from tensorflow.keras.models  import load_model  # TensorFlow is required for Keras to work
from tensorflow.keras.preprocessing import image  # Install pillow instead of PIL
import numpy as np
#load the saved model
model = load_model('/content/drive/MyDrive/model.h5')
#load and preprocess the test image
test_image_path='/content/drive/MyDrive/Brain_Tumor_Detection/train/no/No12.jpg'
img = image.load_img(test_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array,axis=0) #Add batch dimension
img_array /=255. #Normalize the pixel values
#Make predictions
prediction = model.predict(img_array)
#print the prediction
if prediction[0][0] < 0.5:
  print("prediction:No tumor (probability:",prediction[0][0],")")
else:
    print("prediction: Tumor (probability:",prediction[0][0],")")

1/1 [==============================] - 0s 188ms/step
prediction:No tumor (probability: 0.4942709 )
